In [76]:
import pandas as pd

### 1. Study Population
1. 전체 입원 건수(58,976 건)
2. 최초 입원 건수(46,520 명)
3. LOS가 24H 이상(44,770 명)
4. AGE가 18세 이상 89세 미만(35,136 명)

In [12]:
# 입원 기록에 따른 환자별 진단 받은 ICD9 code 합치기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

print("전체 입원 건수: ", len(df1))

전체 입원 건수:  58976


In [13]:
# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기(58,976명 -> 46,520명)
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df1, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("최초 입원 건수: ", len(df5))

최초 입원 건수:  46520


In [14]:
# LOS 추가 후 24H 이상 머문 환자 list만 저장(46,520명 -> 44,770명)
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

LOS가 24H 이상인 환자 수:  44770


In [15]:
# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별(44,770명 -> 37,040명)
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()
print("18세 이상인 환자 수: ", len(subject_list))

18세 이상인 환자 수:  37040


In [16]:
# 89세 이상 환자 제거(37,040명 -> 35,136명)
less = df8["AGE"] < 89
df9 = df8[less].sort_values(by=["AGE"], ascending=True)
subject_list = df9["SUBJECT_ID"].to_list()
print("89세 미만인 환자 수: ", len(subject_list))

89세 미만인 환자 수:  35136


### 2. sepsis-3 환자 사망률 확인

In [77]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["blood poisoning", "Blood poisoning", "Blood Poisoning", "Sepsis", "sepsis"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Sepsis 관련 ICD9 code 수: ", len(disease_list))

df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Sepsis 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"
        
# Heart Disease 진단 받은 환자의 특정 기간 내 사망률
df9.sort_values(by=["DOA"], ascending=False)["DOA"]
print("-----------------------------------------")
print("Sepsis 진단 받은 환자의 특정 기간 내 사망률")
print("Sepsis 진단 받은 환자 수: ", len(final_list))
print("-----------------------------------------")

# 7일 이내 사망
less = df9["DOA"] <= "7days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("7일 이내 사망자 수: ", len(subject_list))
print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 14일 이내 사망
less = df9["DOA"] <= "14days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("14일 이내 사망자 수: ", len(subject_list))
print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 30일 이내 사망
less = df9["DOA"] <= "30days"
df10 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df10["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df9["DOA"] <= "60days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 90일 이내 사망
less = df9["DOA"] <= "90days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("90일 이내 사망자 수: ", len(subject_list))
print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

Sepsis 관련 ICD9 code 수:  6
Sepsis 환자 수(최초 입원):  4781
LOS가 24H 이상인 환자 수:  4593
18세 이상 89세 미만 환자 수:  4058
-----------------------------------------
Sepsis 진단 받은 환자의 특정 기간 내 사망률
Sepsis 진단 받은 환자 수:  4058
-----------------------------------------
7일 이내 사망자 수:  425
7일 이내 사망자 수 비율:  10.473139477575161
-----------------------------------------
14일 이내 사망자 수:  720
14일 이내 사망자 수 비율:  17.742730409068507
-----------------------------------------
30일 이내 사망자 수:  1021
30일 이내 사망자 수 비율:  25.16017742730409
-----------------------------------------
60일 이내 사망자 수:  1147
60일 이내 사망자 수 비율:  28.265155248891077
-----------------------------------------
90일 이내 사망자 수:  1166
90일 이내 사망자 수 비율:  28.7333661902415
-----------------------------------------


### 3. 대표적인 질병 진단 여부에 따른 사망률 비교
-> 2020년 미국질병통제예방센터(CDC)에서 제공한 자료에 따르면 미국인의 사망원인 Top10은 다음과 같다.
1. 심질환 - 
2. 암 - 
3. 사고(의도하지 않은 부상) - ❌
4. 만성 하부 호흡기 질환 - 
5. 뇌졸중(뇌혈관 질환) - 
6. 알츠하이머 - 
7. 당뇨병 - 
8. 신염, 신증후군, 신증 - 
9. 인플루엔자 및 폐렴 - 
10. 고의적 자해(자살) - ❌

##### 3-1. 심질환 환자의 사망률

In [71]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["heart", "Heart"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Heart Disease 관련 ICD9 code 수: ", len(disease_list))

df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Heart Disease 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"
        
# Heart Disease 진단 받은 환자의 특정 기간 내 사망률
df9.sort_values(by=["DOA"], ascending=False)["DOA"]
print("-----------------------------------------")
print("Heart Disease 진단 받은 환자의 특정 기간 내 사망률")
print("Heart Disease 진단 받은 환자 수: ", len(final_list))
print("-----------------------------------------")

# 7일 이내 사망
less = df9["DOA"] <= "7days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("7일 이내 사망자 수: ", len(subject_list))
print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 14일 이내 사망
less = df9["DOA"] <= "14days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("14일 이내 사망자 수: ", len(subject_list))
print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 30일 이내 사망
less = df9["DOA"] <= "30days"
df10 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df10["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df9["DOA"] <= "60days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 90일 이내 사망
less = df9["DOA"] <= "90days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("90일 이내 사망자 수: ", len(subject_list))
print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

Heart Disease 관련 ICD9 code 수:  97
Heart Disease 환자 수(최초 입원):  15730
LOS가 24H 이상인 환자 수:  15319
18세 이상 89세 미만 환자 수:  13991
-----------------------------------------
Heart Disease 진단 받은 환자의 특정 기간 내 사망률
Heart Disease 진단 받은 환자 수:  13991
-----------------------------------------
7일 이내 사망자 수:  614
7일 이내 사망자 수 비율:  4.388535487098849
-----------------------------------------
14일 이내 사망자 수:  1017
14일 이내 사망자 수 비율:  7.268958616253306
-----------------------------------------
30일 이내 사망자 수:  1375
30일 이내 사망자 수 비율:  9.827746408405403
-----------------------------------------
60일 이내 사망자 수:  1498
60일 이내 사망자 수 비율:  10.70688299621185
-----------------------------------------
90일 이내 사망자 수:  1513
90일 이내 사망자 수 비율:  10.814094775212636
-----------------------------------------


##### 3-2 암 환자의 사망률 ❌

In [74]:
# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
# word = ["cancer", "Cancer"]
# disease = '|'.join(word)
# df1 = df[df['LONG_TITLE'].str.contains(disease)]
# disease_list = df1["ICD9_CODE"].to_list()
# print("Cancer 관련 ICD9 code 수: ", len(disease_list))

# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

# def add(group):
#     return ' '.join(group['ICD9_CODE'].fillna(""))

# df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

# mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
# df2 = df1[mask]
# df2

# # 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
# df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
# df4 = pd.merge(df2, df3, on="HADM_ID")
# df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
# df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
# df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

# df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
# df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
# df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
# print("Cancer 환자 수(최초 입원): ", len(df5))

# # LOS 추가 후 24H 이상 머문 환자 list만 저장
# df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
# over = df5["LOS"] >= "1days"
# df6 = df5[over].sort_values(by=["LOS"], ascending=True)
# subject_list = df6["SUBJECT_ID"].to_list()
# print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# # 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
# df = df[["SUBJECT_ID", "DOB"]]
# df['DOB'] = pd.to_datetime(df['DOB']).dt.date
# df7 = pd.merge(df6, df, on="SUBJECT_ID")

# # 첫 입원 시점 연령 칼럼 생성
# df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# # 18세 이상 환자 선별
# over = df7["AGE"] >= 18
# df8 = df7[over].sort_values(by=["AGE"], ascending=True)
# subject_list = df8["SUBJECT_ID"].to_list()

# # 89세 이상 환자 제거
# over = df8["AGE"] < 89
# df9 = df8[over].sort_values(by=["AGE"], ascending=True)
# final_list = df9["SUBJECT_ID"].to_list()
# print("18세 이상 89세 미만 환자 수: ", len(final_list))

# # DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
# for i in range(len(df9)):
#     if df9["DEATHTIME"].iloc[i] != "NaT":
#         df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
#     else:
#         df9["DOA"] = "NaT"
        
# # Heart Disease 진단 받은 환자의 특정 기간 내 사망률
# df9.sort_values(by=["DOA"], ascending=False)["DOA"]
# print("-----------------------------------------")
# print("Cancer 진단 받은 환자의 특정 기간 내 사망률")
# print("Cancer 진단 받은 환자 수: ", len(final_list))
# print("-----------------------------------------")

# # 7일 이내 사망
# less = df9["DOA"] <= "7days"
# df11 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df11["SUBJECT_ID"].to_list()
# print("7일 이내 사망자 수: ", len(subject_list))
# print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 14일 이내 사망
# less = df9["DOA"] <= "14days"
# df12 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df12["SUBJECT_ID"].to_list()
# print("14일 이내 사망자 수: ", len(subject_list))
# print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 30일 이내 사망
# less = df9["DOA"] <= "30days"
# df10 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df10["SUBJECT_ID"].to_list()
# print("30일 이내 사망자 수: ", len(subject_list))
# print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 60일 이내 사망
# less = df9["DOA"] <= "60days"
# df11 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df11["SUBJECT_ID"].to_list()
# print("60일 이내 사망자 수: ", len(subject_list))
# print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 90일 이내 사망
# less = df9["DOA"] <= "90days"
# df12 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df12["SUBJECT_ID"].to_list()
# print("90일 이내 사망자 수: ", len(subject_list))
# print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

##### 3-4. 만성 하부 호흡기 질환 환자의 사망률

In [70]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["respiratory", "Respiratory"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Respiratory 관련 ICD9 code 수: ", len(disease_list))

df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]
df2

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Respiratory 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"
        
# Heart Disease 진단 받은 환자의 특정 기간 내 사망률
df9.sort_values(by=["DOA"], ascending=False)["DOA"]
print("-----------------------------------------")
print("Respiratory 진단 받은 환자의 특정 기간 내 사망률")
print("Respiratory 진단 받은 환자 수: ", len(final_list))
print("-----------------------------------------")

# 7일 이내 사망
less = df9["DOA"] <= "7days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("7일 이내 사망자 수: ", len(subject_list))
print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 14일 이내 사망
less = df9["DOA"] <= "14days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("14일 이내 사망자 수: ", len(subject_list))
print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 30일 이내 사망
less = df9["DOA"] <= "30days"
df10 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df10["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df9["DOA"] <= "60days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 90일 이내 사망
less = df9["DOA"] <= "90days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("90일 이내 사망자 수: ", len(subject_list))
print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

Respiratory 관련 ICD9 code 수:  92
Respiratory 환자 수(최초 입원):  10329
LOS가 24H 이상인 환자 수:  9932
18세 이상 89세 미만 환자 수:  7631
-----------------------------------------
Respiratory 진단 받은 환자의 특정 기간 내 사망률
Respiratory 진단 받은 환자 수:  7631
-----------------------------------------
7일 이내 사망자 수:  782
7일 이내 사망자 수 비율:  10.24767396147294
-----------------------------------------
14일 이내 사망자 수:  1285
14일 이내 사망자 수 비율:  16.83920849167868
-----------------------------------------
30일 이내 사망자 수:  1742
30일 이내 사망자 수 비율:  22.82793867120954
-----------------------------------------
60일 이내 사망자 수:  1905
60일 이내 사망자 수 비율:  24.963962783383568
-----------------------------------------
90일 이내 사망자 수:  1932
90일 이내 사망자 수 비율:  25.317782728344906
-----------------------------------------


##### 3-5. 뇌졸중 환자의 사망률

In [69]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["stroke", "brain", "cerebrovascular", "Stroke", "Brain", "Cerebrovascular"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Stroke 관련 ICD9 code 수: ", len(disease_list))

df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]
df2

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Stroke 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"
        
# Heart Disease 진단 받은 환자의 특정 기간 내 사망률
df9.sort_values(by=["DOA"], ascending=False)["DOA"]
print("-----------------------------------------")
print("Stroke 진단 받은 환자의 특정 기간 내 사망률")
print("Stroke 진단 받은 환자 수: ", len(final_list))
print("-----------------------------------------")

# 7일 이내 사망
less = df9["DOA"] <= "7days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("7일 이내 사망자 수: ", len(subject_list))
print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 14일 이내 사망
less = df9["DOA"] <= "14days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("14일 이내 사망자 수: ", len(subject_list))
print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 30일 이내 사망
less = df9["DOA"] <= "30days"
df10 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df10["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df9["DOA"] <= "60days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 90일 이내 사망
less = df9["DOA"] <= "90days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("90일 이내 사망자 수: ", len(subject_list))
print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

Stroke 관련 ICD9 code 수:  114
Stroke 환자 수(최초 입원):  5234
LOS가 24H 이상인 환자 수:  5017
18세 이상 89세 미만 환자 수:  4642
-----------------------------------------
Stroke 진단 받은 환자의 특정 기간 내 사망률
Stroke 진단 받은 환자 수:  4642
-----------------------------------------
7일 이내 사망자 수:  412
7일 이내 사망자 수 비율:  8.875484704868592
-----------------------------------------
14일 이내 사망자 수:  605
14일 이내 사망자 수 비율:  13.033175355450238
-----------------------------------------
30일 이내 사망자 수:  727
30일 이내 사망자 수 비율:  15.661352865144334
-----------------------------------------
60일 이내 사망자 수:  754
60일 이내 사망자 수 비율:  16.242998707453683
-----------------------------------------
90일 이내 사망자 수:  758
90일 이내 사망자 수 비율:  16.329168461869884
-----------------------------------------


##### 3-6. 알츠하이머 환자의 사망률 ❌

In [61]:
# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
# word = ["alzheimer"]
# disease = '|'.join(word)
# df1 = df[df['LONG_TITLE'].str.contains(disease)]
# disease_list = df1["ICD9_CODE"].to_list()
# print("Alzheimer 관련 ICD9 code 수: ", len(disease_list))

# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

# def add(group):
#     return ' '.join(group['ICD9_CODE'].fillna(""))

# df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

# mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
# df2 = df1[mask]
# df2

# # 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
# df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
# df4 = pd.merge(df2, df3, on="HADM_ID")
# df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
# df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
# df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

# df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
# df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
# df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
# print("Alzheimer 환자 수(최초 입원): ", len(df5))

# # LOS 추가 후 24H 이상 머문 환자 list만 저장
# df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
# over = df5["LOS"] >= "1days"
# df6 = df5[over].sort_values(by=["LOS"], ascending=True)
# subject_list = df6["SUBJECT_ID"].to_list()
# print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# # 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
# df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
# df = df[["SUBJECT_ID", "DOB"]]
# df['DOB'] = pd.to_datetime(df['DOB']).dt.date
# df7 = pd.merge(df6, df, on="SUBJECT_ID")

# # 첫 입원 시점 연령 칼럼 생성
# df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# # 18세 이상 환자 선별
# over = df7["AGE"] >= 18
# df8 = df7[over].sort_values(by=["AGE"], ascending=True)
# subject_list = df8["SUBJECT_ID"].to_list()

# # 89세 이상 환자 제거
# over = df8["AGE"] < 89
# df9 = df8[over].sort_values(by=["AGE"], ascending=True)
# final_list = df9["SUBJECT_ID"].to_list()
# print("18세 이상 89세 미만 환자 수: ", len(final_list))

# # DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
# for i in range(len(df9)):
#     if df9["DEATHTIME"].iloc[i] != "NaT":
#         df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
#     else:
#         df9["DOA"] = "NaT"
        
# # Heart Disease 진단 받은 환자의 특정 기간 내 사망률
# df9.sort_values(by=["DOA"], ascending=False)["DOA"]
# print("-----------------------------------------")
# print("Alzheimer 진단 받은 환자의 특정 기간 내 사망률")
# print("Alzheimer 진단 받은 환자 수: ", len(final_list))
# print("-----------------------------------------")

# # 7일 이내 사망
# less = df9["DOA"] <= "7days"
# df11 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df11["SUBJECT_ID"].to_list()
# print("7일 이내 사망자 수: ", len(subject_list))
# print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 14일 이내 사망
# less = df9["DOA"] <= "14days"
# df12 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df12["SUBJECT_ID"].to_list()
# print("14일 이내 사망자 수: ", len(subject_list))
# print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 30일 이내 사망
# less = df9["DOA"] <= "30days"
# df10 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df10["SUBJECT_ID"].to_list()
# print("30일 이내 사망자 수: ", len(subject_list))
# print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 60일 이내 사망
# less = df9["DOA"] <= "60days"
# df11 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df11["SUBJECT_ID"].to_list()
# print("60일 이내 사망자 수: ", len(subject_list))
# print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

# # 90일 이내 사망
# less = df9["DOA"] <= "90days"
# df12 = df9[less].sort_values(by=["DOA"], ascending=True)
# subject_list = df12["SUBJECT_ID"].to_list()
# print("90일 이내 사망자 수: ", len(subject_list))
# print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
# print("-----------------------------------------")

##### 3-7. 당뇨병 환자의 사망률

In [68]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["diabet", "Diabet"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Diabetes 관련 ICD9 code 수: ", len(disease_list))

df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]
df2

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Diabetes 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"
        
# Diabetes 진단 받은 환자의 특정 기간 내 사망률
df9.sort_values(by=["DOA"], ascending=False)["DOA"]
print("-----------------------------------------")
print("Diabetes 진단 받은 환자의 특정 기간 내 사망률")
print("Diabetes 진단 받은 환자 수: ", len(final_list))
print("-----------------------------------------")

# 7일 이내 사망
less = df9["DOA"] <= "7days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("7일 이내 사망자 수: ", len(subject_list))
print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 14일 이내 사망
less = df9["DOA"] <= "14days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("14일 이내 사망자 수: ", len(subject_list))
print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 30일 이내 사망
less = df9["DOA"] <= "30days"
df10 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df10["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df9["DOA"] <= "60days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 90일 이내 사망
less = df9["DOA"] <= "90days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("90일 이내 사망자 수: ", len(subject_list))
print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

Diabetes 관련 ICD9 code 수:  84
Diabetes 환자 수(최초 입원):  10793
LOS가 24H 이상인 환자 수:  10487
18세 이상 89세 미만 환자 수:  9845
-----------------------------------------
Diabetes 진단 받은 환자의 특정 기간 내 사망률
Diabetes 진단 받은 환자 수:  9845
-----------------------------------------
7일 이내 사망자 수:  415
7일 이내 사망자 수 비율:  4.215337734890808
-----------------------------------------
14일 이내 사망자 수:  627
14일 이내 사망자 수 비율:  6.368715083798883
-----------------------------------------
30일 이내 사망자 수:  805
30일 이내 사망자 수 비율:  8.176739461655663
-----------------------------------------
60일 이내 사망자 수:  849
60일 이내 사망자 수 비율:  8.623666835957339
-----------------------------------------
90일 이내 사망자 수:  862
90일 이내 사망자 수 비율:  8.755713560182835
-----------------------------------------


##### 3-8. 신염, 신증후군, 신증 환자의 사망률

In [67]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["Nephritis", "Nephrotic", "Nephropathy", "nephritis", "nephrotic", "nephropathy"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Nephrotic Syndrome 관련 ICD9 code 수: ", len(disease_list))

df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]
df2

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Nephrotic Syndrome 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"
        
# Nephrotic Syndrome 진단 받은 환자의 특정 기간 내 사망률
df9.sort_values(by=["DOA"], ascending=False)["DOA"]
print("-----------------------------------------")
print("Nephrotic Syndrome 진단 받은 환자의 특정 기간 내 사망률")
print("Nephrotic Syndrome 진단 받은 환자 수: ", len(final_list))
print("-----------------------------------------")

# 7일 이내 사망
less = df9["DOA"] <= "7days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("7일 이내 사망자 수: ", len(subject_list))
print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 14일 이내 사망
less = df9["DOA"] <= "14days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("14일 이내 사망자 수: ", len(subject_list))
print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 30일 이내 사망
less = df9["DOA"] <= "30days"
df10 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df10["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df9["DOA"] <= "60days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 90일 이내 사망
less = df9["DOA"] <= "90days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("90일 이내 사망자 수: ", len(subject_list))
print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

Nephrotic Syndrome 관련 ICD9 code 수:  43
Nephrotic Syndrome 환자 수(최초 입원):  3099
LOS가 24H 이상인 환자 수:  3050
18세 이상 89세 미만 환자 수:  2916
-----------------------------------------
Nephrotic Syndrome 진단 받은 환자의 특정 기간 내 사망률
Nephrotic Syndrome 진단 받은 환자 수:  2916
-----------------------------------------
7일 이내 사망자 수:  73
7일 이내 사망자 수 비율:  2.5034293552812072
-----------------------------------------
14일 이내 사망자 수:  128
14일 이내 사망자 수 비율:  4.38957475994513
-----------------------------------------
30일 이내 사망자 수:  179
30일 이내 사망자 수 비율:  6.138545953360768
-----------------------------------------
60일 이내 사망자 수:  195
60일 이내 사망자 수 비율:  6.68724279835391
-----------------------------------------
90일 이내 사망자 수:  200
90일 이내 사망자 수 비율:  6.858710562414267
-----------------------------------------


##### 3-9. 인플루엔자 및 폐렴 환자의 사망률

In [75]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["influenza","Influenza", "Pneumonia", "pneumonia"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Pneumonia 관련 ICD9 code 수: ", len(disease_list))

df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]
df2

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])

df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Pneumonia 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"
        
# Heart Disease 진단 받은 환자의 특정 기간 내 사망률
df9.sort_values(by=["DOA"], ascending=False)["DOA"]
print("-----------------------------------------")
print("Pneumonia 진단 받은 환자의 특정 기간 내 사망률")
print("Pneumonia 진단 받은 환자 수: ", len(final_list))
print("-----------------------------------------")

# 7일 이내 사망
less = df9["DOA"] <= "7days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("7일 이내 사망자 수: ", len(subject_list))
print("7일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 14일 이내 사망
less = df9["DOA"] <= "14days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("14일 이내 사망자 수: ", len(subject_list))
print("14일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 30일 이내 사망
less = df9["DOA"] <= "30days"
df10 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df10["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df9["DOA"] <= "60days"
df11 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 90일 이내 사망
less = df9["DOA"] <= "90days"
df12 = df9[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("90일 이내 사망자 수: ", len(subject_list))
print("90일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

Pneumonia 관련 ICD9 code 수:  76
Pneumonia 환자 수(최초 입원):  8776
LOS가 24H 이상인 환자 수:  8585
18세 이상 89세 미만 환자 수:  7914
-----------------------------------------
Pneumonia 진단 받은 환자의 특정 기간 내 사망률
Pneumonia 진단 받은 환자 수:  7914
-----------------------------------------
7일 이내 사망자 수:  591
7일 이내 사망자 수 비율:  7.467778620166793
-----------------------------------------
14일 이내 사망자 수:  1003
14일 이내 사망자 수 비율:  12.673742734394743
-----------------------------------------
30일 이내 사망자 수:  1374
30일 이내 사망자 수 비율:  17.36163760424564
-----------------------------------------
60일 이내 사망자 수:  1514
60일 이내 사망자 수 비율:  19.130654536264846
-----------------------------------------
90일 이내 사망자 수:  1532
90일 이내 사망자 수 비율:  19.3580995703816
-----------------------------------------
